# http://webgraphviz.com

In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
import numpy as np
from sklearn.metrics import roc_auc_score

%matplotlib inline

file=r'/Users/census_income.csv'

ci_train=pd.read_csv(file)
ci_train.drop(['education'],axis=1,inplace=True)
ci_train['Y'].value_counts().index
ci_train['Y']=(ci_train['Y']==' >50K').astype(int)
cat_cols=ci_train.select_dtypes(['object']).columns
cat_cols
ci_train.shape
for col in cat_cols:
    freqs=ci_train[col].value_counts()
    k=freqs.index[freqs>500][:-1]
    for cat in k:
        name=col+'_'+cat
        ci_train[name]=(ci_train[col]==cat).astype(int)
    del ci_train[col]
    print(col)
ci_train.isnull().sum()
x_train=ci_train.drop(['Y'],1)
y_train=ci_train['Y']

from sklearn.model_selection import RandomizedSearchCV
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
       }
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
random_search=RandomizedSearchCV(clf,cv=10,
                                 param_distributions=params,
                                 scoring='roc_auc',
                                 n_iter=10
                                    )
random_search.fit(x_train,y_train)
random_search.best_estimator_

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

report(random_search.cv_results_,5)
dtree=random_search.best_estimator_

dtree.fit(x_train,y_train)



In [ ]:
dotfile = open("mytree.dot", 'w')

tree.export_graphviz(dtree,out_file=dotfile,
                     feature_names=x_train.columns,
                    class_names=["0","1"],
                     proportion=True)
dotfile.close()

In [ ]:
# Random Forest


from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }


n_iter_search = 10

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='roc_auc',cv=5)
random_search.fit(x_train, y_train)


rf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=50, 
                          max_features=10, max_leaf_nodes=None, min_impurity_split=1e-07, 
                          min_samples_leaf=10, min_samples_split=20, min_weight_fraction_leaf=0.0, 
                          n_estimators=300, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False)


rf.fit(x_train,y_train)


feat_imp_df=pd.DataFrame({'features':x_train.columns,'importance':rf.feature_importances_})

feat_imp_df.sort_values('importance',ascending=False)

var_name='Te'

preds=rf.predict_proba(x_train)[:,1]

var_data=pd.DataFrame({'var':x_train[var_name],'response':preds})